In [57]:
from chakraview import ChakraView
ck = ChakraView()
spot_df = ck.get_spot_df('nifty')
spot_df

,column00,column0,c,date,h,l,oi,o,symbol,time,v,strike,right
0,0,9969,12215.10,2020-01-01,12216.25,12201.55,<NA>,12202.15,NIFTY,09:15:00,0,0,TY
1,1,9970,12206.80,2020-01-01,12216.45,12205.70,<NA>,12215.45,NIFTY,09:16:00,0,0,TY
2,2,9971,12202.70,2020-01-01,12207.40,12202.70,<NA>,12206.50,NIFTY,09:17:00,0,0,TY
3,3,9972,12207.85,2020-01-01,12208.40,12201.20,<NA>,12202.15,NIFTY,09:18:00,0,0,TY
4,4,9973,12212.45,2020-01-01,12213.20,12207.70,<NA>,12208.35,NIFTY,09:19:00,0,0,TY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
464051,464051,45805,23656.40,2024-12-31,23656.40,23646.50,<NA>,23648.70,NIFTY,15:25:00,0,0,TY
464052,464052,45806,23657.65,2024-12-31,23663.30,23655.75,<NA>,23655.75,NIFTY,15:26:00,0,0,TY
464053,464053,45807,23658.75,2024-12-31,23659.80,23651.95,<NA>,23657.45,NIFTY,15:27:00,0,0,TY
464054,464054,45808,23658.80,2024-12-31,23664.05,23655.05,<NA>,23658.90,NIFTY,15:28:00,0,0,TY


In [69]:
import datetime
date1 = datetime.date(2024, 12, 31)
time1 = datetime.time(9, 30, 0)
day = spot_df[(spot_df['date'] == date1) & (spot_df['time'] == time1)]
close = day['c'].iloc[0]
close

np.float64(23546.5)

In [70]:
call = ck.find_ticker_by_moneyness(date1, time1, close, 50, 'CE', 0)
put = ck.find_ticker_by_moneyness(date1, time1, close, 50, 'PE', 0)
print(call)
print(put)

Elapsed Time In Getting All Ticks: 0.009588241577148438
Elapsed Time In Getting Moneyness Details: 0.013483524322509766
Elapsed Time In Getting All Ticks: 0.011215686798095703
Elapsed Time In Getting Moneyness Details: 0.01536107063293457
{'symbol': 'NIFTY02JAN2523550CE', 'o': 129.45, 'h': 137.2, 'l': 129.45, 'c': 135.0, 'v': 518175, 'oi': 2237550.0, 'strike': 23550, 'right': 'CE'}
{'symbol': 'NIFTY02JAN2523550PE', 'o': 125.05, 'h': 125.35, 'l': 116.5, 'c': 119.45, 'v': 767625, 'oi': 2582625.0, 'strike': 23550, 'right': 'PE'}


In [71]:
tsdf = ck.get_all_ticks_by_timestamp(date1, time1)
tsdf

Elapsed Time In Getting All Ticks: 0.010139226913452148


,symbol,o,h,l,c,v,oi,strike,right
0,NIFTY02JAN2521600PE,0.7500,0.8000,0.7500,0.80,36075,10516800.0,21600,PE
1,NIFTY02JAN2521650PE,0.9000,0.9000,0.8500,0.90,8400,429525.0,21650,PE
2,NIFTY02JAN2521700PE,0.9500,0.9500,0.9500,0.95,2850,786900.0,21700,PE
3,NIFTY02JAN2521750PE,1.0500,1.0500,1.0000,1.05,11250,243675.0,21750,PE
4,NIFTY02JAN2521800PE,1.1000,1.1000,1.1000,1.10,3900,885900.0,21800,PE
...,...,...,...,...,...,...,...,...,...
124,NIFTY02JAN2526400CE,0.9000,0.9000,0.9000,0.90,56850,1950300.0,26400,CE
125,NIFTY02JAN2526450CE,0.8500,0.8500,0.8500,0.85,600,739500.0,26450,CE
126,NIFTY02JAN2526500CE,0.8000,0.8500,0.8000,0.80,27750,8799900.0,26500,CE
127,NIFTY-I,23677.0508,23696.9492,23676.0508,23694.00,39850,12438475.0,0,-I


In [84]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import datetime

# ---------------------------
# Black-Scholes Pricing
# ---------------------------
def bs_price(opt_type, S, K, r, sigma, T):
    if T <= 0 or sigma <= 0:
        return np.nan
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if opt_type == "CE":
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)


# ---------------------------
# Implied Volatility Solver
# ---------------------------
def solve_iv(opt_type, S, K, r, T, market_price, tol=1e-6, max_iter=100):
    sigma = 0.20  # Start guess

    for _ in range(max_iter):

        price = bs_price(opt_type, S, K, r, sigma, T)
        if np.isnan(price):
            return np.nan

        # Vega
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        if vega < 1e-8:
            return np.nan

        diff = price - market_price
        sigma_new = sigma - diff / vega

        if abs(sigma_new - sigma) < tol:
            return max(sigma_new, 1e-6)

        sigma = sigma_new

    return np.nan


# ---------------------------
# Main Function
# ---------------------------
def compute_iv_delta(tsdf: pd.DataFrame, date, r: float = 0.07):
    # --------------------------------------------------------
    # 1. Extract Spot Price (use "NIFTY" row)
    # --------------------------------------------------------
    spot = tsdf.loc[tsdf["symbol"] == "NIFTY", "c"].iloc[0]

    # --------------------------------------------------------
    # 2. Parse Expiry from symbols
    # --------------------------------------------------------
    def extract_expiry(sym: str):
        # Option symbols look like: NIFTY02JAN2521600PE
        # Extract substring at positions 5 to 12 → "02JAN25"
        if len(sym) >= 12 and sym.startswith("NIFTY"):
            part = sym[5:12]  # DDMMMYY
            try:
                return datetime.datetime.strptime(part, "%d%b%y")
            except:
                return pd.NaT
        return pd.NaT

    tsdf["expiry"] = tsdf["symbol"].apply(extract_expiry)

    # Convert to pandas datetime explicitly
    tsdf["expiry"] = pd.to_datetime(tsdf["expiry"], errors="coerce")

    # --------------------------------------------------------
    # 3. Compute time to maturity
    # --------------------------------------------------------
    today = pd.to_datetime(date)
    tsdf["ttm"] = (tsdf["expiry"] - today).dt.days / 365.0

    # --------------------------------------------------------
    # 4. Compute IV
    # --------------------------------------------------------
    ivs = []

    for _, row in tsdf.iterrows():

        if row["right"] not in ("CE", "PE"):
            ivs.append(np.nan)
            continue

        iv = solve_iv(
            opt_type=row["right"],
            S=spot,
            K=row["strike"],
            r=r,
            T=row["ttm"],
            market_price=row["c"]
        )
        ivs.append(iv)

    print('IVs ', ivs)
    tsdf["iv"] = ivs

    # --------------------------------------------------------
    # 5. Compute Delta
    # --------------------------------------------------------
    deltas = []

    for _, row in tsdf.iterrows():

        if row["right"] not in ("CE", "PE"):
            deltas.append(np.nan)
            continue

        S = spot
        K = row["strike"]
        sigma = row["iv"]
        T = row["ttm"]

        if pd.isna(sigma) or sigma <= 0 or T <= 0:
            deltas.append(np.nan)
            continue

        d1 = (np.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*np.sqrt(T))

        if row["right"] == "CE":
            delta = norm.cdf(d1)
        else:
            delta = norm.cdf(d1) - 1

        deltas.append(delta)

    tsdf["delta"] = deltas
    print(f'Deltas ', deltas)

    return tsdf


In [ ]:
def find_ticker_by_delta(date, time, delta, right):
    tsdf = ck.get_all_ticks_by_timestamp(date, time)
    delta_df = compute_iv_delta(tsdf, date)
    if right.upper() == 'CE':
        call_df = delta_df[delta_df['right'] == 'CE']
        call_df['delta_diff'] = (call_df['delta'] - delta).abs()
        best_row = call_df.loc[call_df['delta_diff'].idxmin()]
        delta_dict = {
            'symbol': best_row['symbol'],
            'o': best_row['o'],
            'h': best_row['h'],
            'l': best_row['l'],
            'c': best_row['c'],
            'v': best_row['v'],
            'oi': best_row['oi'],
            'strike': best_row['strike'],
            'right': best_row['right']
        }
        return delta_dict
    elif right.upper() == 'PE':
        put_df = delta_df[delta_df['right'] == 'PE']
        put_df = put_df.copy()
        put_df['delta'] = put_df['delta'].abs()
        put_df['delta_diff'] = (put_df['delta'] - delta).abs()
        best_row = put_df.loc[put_df['delta_diff'].idxmin()]
        delta_dict = {
            'symbol': best_row['symbol'],
            'o': best_row['o'],
            'h': best_row['h'],
            'l': best_row['l'],
            'c': best_row['c'],
            'v': best_row['v'],
            'oi': best_row['oi'],
            'strike': best_row['strike'],
            'right': best_row['right']
        }
        return delta_dict

def find_ticker_by_strike(date, time, strike, right):
    timestampdf = ck.get_all_ticks_by_timestamp(date, time)
    filtered_df = timestampdf[(timestampdf['strike'] == strike) & timestampdf['right'] == right]
    dict_to_return = {
            'symbol': filtered_df['symbol'],
            'o': filtered_df['o'],
            'h': filtered_df['h'],
            'l': filtered_df['l'],
            'c': filtered_df['c'],
            'v': filtered_df['v'],
            'oi': filtered_df['oi']
        }
    return dict_to_return



date=datetime.date(2024, 12, 31)
time=datetime.time(9, 20, 0)
tsdf = ck.get_all_ticks_by_timestamp(date, time)
print(tsdf)
delta_df = compute_iv_delta(tsdf, date)
print(delta_df)



Elapsed Time In Getting All Ticks: 0.008963346481323242
                  symbol         o           h         l         c       v  \
0    NIFTY02JAN2521600PE      0.70      0.7000      0.70      0.70  114675   
1    NIFTY02JAN2521650PE      0.75      0.8000      0.75      0.80    2700   
2    NIFTY02JAN2521700PE      0.90      0.9500      0.90      0.95    4875   
3    NIFTY02JAN2521750PE      1.00      1.0500      1.00      1.00   15525   
4    NIFTY02JAN2521800PE      1.05      1.0500      1.00      1.05   10500   
..                   ...       ...         ...       ...       ...     ...   
128  NIFTY02JAN2526400CE      0.90      0.9000      0.90      0.90   12450   
129  NIFTY02JAN2526450CE      0.90      0.9500      0.90      0.95    8325   
130  NIFTY02JAN2526500CE      0.90      0.9500      0.90      0.95  462750   
131              NIFTY-I  23692.00  23700.0508  23686.00  23698.25   49025   
132                NIFTY  23554.50  23564.0000  23542.80  23562.40       0   

       

In [133]:
import datetime
# atm_call = ck.find_ticker_by_moneyness(datetime.date(2024, 12, 31), datetime.time(9,30,0), 23562.40, 50, 'CE', 0)
# atm_put = ck.find_ticker_by_moneyness(datetime.date(2024, 12, 31), datetime.time(9, 30, 0), 23562.40, 50, 'PE', 0)
atm_call = find_ticker_by_delta(datetime.date(2024, 12, 31), datetime.time(9, 30, 0), 0.25, 'CE')
atm_put = find_ticker_by_delta(datetime.date(2024, 12, 31), datetime.time(9, 30, 0), 0.25, 'PE')
protection_call = find_ticker_by_delta(datetime.date(2024, 12, 31), datetime.time(9, 30, 0), 0.05, 'CE')
protection_put = find_ticker_by_delta(datetime.date(2024, 12, 31), datetime.time(9, 30, 0), 0.05, 'PE')
protection_call_strike = protection_call.get('strike')

Elapsed Time In Getting All Ticks: 0.010450124740600586
IVs  [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, np.float64(0.33528867085908237), np.float64(0.3264961778758716), np.float64(0.3163286765439916), np.float64(0.30922440755711617), np.float64(0.2996691906302203), np.float64(0.29080648855662955), np.float64(0.3846690254825681), np.float64(0.2816243226982047), np.float64(0.2729043913775988), np.float64(0.2637855787608312), np.float64(0.25429648624691925), np.float64(0.24445897379670736), np.float64(0.23590947838415977), np.float64(0.2272431872954607), np.float64(0.2192215387107029), np.float64(0.21161321089794236), np.float64(0.21147906881530296), np.float64(0.2558622237379194), np.float64(0.20563487390827215), np.float64(0.24435228280975954), np.float64(0.20037116018918102), np.float64(0.19595615698958965), np.float64(0.23467145856644517), np.float64(0.19063168244265186), np.float64(0.2192450087574686), np.float64(0.18734804708182573), np.float64(0.20769805009278336)

C:\Users\admin\AppData\Local\Temp\ipykernel_5692\2556239735.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



IVs  [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, np.float64(0.33528867085908237), np.float64(0.3264961778758716), np.float64(0.3163286765439916), np.float64(0.30922440755711617), np.float64(0.2996691906302203), np.float64(0.29080648855662955), np.float64(0.3846690254825681), np.float64(0.2816243226982047), np.float64(0.2729043913775988), np.float64(0.2637855787608312), np.float64(0.25429648624691925), np.float64(0.24445897379670736), np.float64(0.23590947838415977), np.float64(0.2272431872954607), np.float64(0.2192215387107029), np.float64(0.21161321089794236), np.float64(0.21147906881530296), np.float64(0.2558622237379194), np.float64(0.20563487390827215), np.float64(0.24435228280975954), np.float64(0.20037116018918102), np.float64(0.19595615698958965), np.float64(0.23467145856644517), np.float64(0.19063168244265186), np.float64(0.2192450087574686), np.float64(0.18734804708182573), np.float64(0.20769805009278336), np.float64(0.1844521176551484), np.float64(0.185768575

C:\Users\admin\AppData\Local\Temp\ipykernel_5692\2556239735.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



IVs  [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, np.float64(0.33528867085908237), np.float64(0.3264961778758716), np.float64(0.3163286765439916), np.float64(0.30922440755711617), np.float64(0.2996691906302203), np.float64(0.29080648855662955), np.float64(0.3846690254825681), np.float64(0.2816243226982047), np.float64(0.2729043913775988), np.float64(0.2637855787608312), np.float64(0.25429648624691925), np.float64(0.24445897379670736), np.float64(0.23590947838415977), np.float64(0.2272431872954607), np.float64(0.2192215387107029), np.float64(0.21161321089794236), np.float64(0.21147906881530296), np.float64(0.2558622237379194), np.float64(0.20563487390827215), np.float64(0.24435228280975954), np.float64(0.20037116018918102), np.float64(0.19595615698958965), np.float64(0.23467145856644517), np.float64(0.19063168244265186), np.float64(0.2192450087574686), np.float64(0.18734804708182573), np.float64(0.20769805009278336), np.float64(0.1844521176551484), np.float64(0.185768575

In [134]:

def calculate_payoff():
    protection_call_strike = protection_call.get('strike')
    protection_put_strike = protection_put.get('strike')
    short_strike_call = atm_call.get('strike')
    short_strike_put = atm_put.get('strike')
    short_entry_call = atm_call.get('c')
    short_entry_put = atm_put.get('c')
    protection_entry_call = protection_call.get('c')
    protection_entry_put = protection_put.get('c')
    payoff_list = []
    # starting point = 50 pts below protection put strike
    start_nifty = protection_put_strike - 50

    # end point = you likely want 50 points above protection call
    end_nifty = protection_call_strike + 50

    payoff_list = []

    price = start_nifty
    while price <= end_nifty:
        updated_short_call_price = price - short_strike_call
        updated_short_call_price = updated_short_call_price if updated_short_call_price > 0 else 0
        updated_short_put_price = short_strike_put - price
        updated_short_put_price = updated_short_put_price if updated_short_put_price > 0 else 0
        updated_long_call_price = price - protection_call_strike
        updated_long_call_price = updated_long_call_price if updated_long_call_price > 0 else 0
        updated_long_put_price  = protection_put_strike - price
        updated_long_put_price = updated_long_put_price if updated_long_put_price > 0 else 0

        short_call_pl = (short_entry_call - updated_short_call_price) * 75
        long_call_pl = (updated_long_call_price - protection_entry_call) * 75
        short_put_pl = (short_entry_put - updated_short_put_price) * 75
        long_put_pl = (updated_long_put_price - protection_entry_put) * 75
        total_pl = short_call_pl + long_call_pl + short_put_pl + long_put_pl
        payoff_list.append({'underlying': price, 'P/L': total_pl})

        price += 1      

    return payoff_list



In [135]:
import plotly.graph_objects as go
import pandas as pd

payoff_list = calculate_payoff()
payoff_df = pd.DataFrame(payoff_list)

# Extract breakeven points
# We find points where P/L crosses zero
payoff_df = payoff_df.sort_values("underlying")
payoff_df["sign"] = payoff_df["P/L"].apply(lambda x: 1 if x >= 0 else -1)
payoff_df["sign_shift"] = payoff_df["sign"].shift()

breakevens = []
for i in range(1, len(payoff_df)):
    if payoff_df.loc[i, "sign"] != payoff_df.loc[i - 1, "sign"]:
        # Linear interpolation for exact break-even point
        x1, y1 = payoff_df.loc[i - 1, ["underlying", "P/L"]]
        x2, y2 = payoff_df.loc[i, ["underlying", "P/L"]]
        be = x1 + (0 - y1) * (x2 - x1) / (y2 - y1)
        breakevens.append(be)

# Main payoff line
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=payoff_df['underlying'],
    y=payoff_df['P/L'],
    mode='lines',
    line=dict(color='blue', width=2),
    name="Payoff"
))

# ---------------------------------------------------------
# Add green/red shading using filled area traces
# ---------------------------------------------------------
fig.add_trace(go.Scatter(
    x=payoff_df['underlying'],
    y=payoff_df['P/L'].clip(lower=0),
    fill='tozeroy',
    mode='none',
    fillcolor='rgba(0, 200, 0, 0.3)',
    name="Profit zone"
))

fig.add_trace(go.Scatter(
    x=payoff_df['underlying'],
    y=payoff_df['P/L'].clip(upper=0),
    fill='tozeroy',
    mode='none',
    fillcolor='rgba(200, 0, 0, 0.3)',
    name="Loss zone"
))

# ---------------------------------------------------------
# Add break-even vertical dotted lines
# ---------------------------------------------------------
for be in breakevens:
    fig.add_vline(
        x=be,
        line=dict(color="blue", width=2, dash="dot"),
        annotation_text=f"BE: {round(be,2)}",
        annotation_position="top"
    )

fig.update_layout(
    title="Payoff Diagram",
    xaxis_title="Underlying",
    yaxis_title="P/L",
    template="plotly_white",
    showlegend=True
)

fig.show()

print("Max Profit:", payoff_df['P/L'].max())
print("Max Loss:", payoff_df['P/L'].min())
print("Breakeven Points:", breakevens)


Max Profit: 5820.0
Max Loss: -20430.0
Breakeven Points: [23272.4, 23877.6]


In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import datetime

# ---------------------------
# Black-Scholes Pricing
# ---------------------------
def bs_price(opt_type, S, K, r, sigma, T):
    if T <= 0 or sigma <= 0:
        return np.nan
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if opt_type == "CE":
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)


# ---------------------------
# Implied Volatility Solver
# ---------------------------
def solve_iv(opt_type, S, K, r, T, market_price, tol=1e-6, max_iter=100):
    sigma = 0.20  # Start guess

    for _ in range(max_iter):

        price = bs_price(opt_type, S, K, r, sigma, T)
        if np.isnan(price):
            return np.nan

        # Vega
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)
        if vega < 1e-8:
            return np.nan

        diff = price - market_price
        sigma_new = sigma - diff / vega

        if abs(sigma_new - sigma) < tol:
            return max(sigma_new, 1e-6)

        sigma = sigma_new

    return np.nan
